In [1]:
pip install transformers

In [2]:
import pandas as pd
import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from transformers import BertTokenizer, BertModel

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Load the expert and crowd annotations data
expert_annotations_path = '/content/drive/My Drive/ExpertAnnotations.tsv'
crowd_annotations_path = '/content/drive/My Drive/CrowdAnnotations.tsv'

# Read the data
expert_data = pd.read_csv(expert_annotations_path, sep='\t', header=None)
crowd_data = pd.read_csv(crowd_annotations_path, sep='\t', header=None)

# Display the first few rows of the data
print(expert_data.head())
print()
print(crowd_data.head())

                           0                            1  2  3  4
0  1056338697_4f7d7ce270.jpg  2549968784_39bfbe44f9.jpg#2  1  1  1
1  1056338697_4f7d7ce270.jpg  2718495608_d8533e3ac5.jpg#2  1  1  2
2  1056338697_4f7d7ce270.jpg  3181701312_70a379ab6e.jpg#2  1  1  2
3  1056338697_4f7d7ce270.jpg  3207358897_bfa61fa3c6.jpg#2  1  2  2
4  1056338697_4f7d7ce270.jpg  3286822339_5535af6b93.jpg#2  1  1  2

                           0                            1    2  3  4
0  1056338697_4f7d7ce270.jpg  1056338697_4f7d7ce270.jpg#2  1.0  3  0
1  1056338697_4f7d7ce270.jpg   114051287_dd85625a04.jpg#2  0.0  0  3
2  1056338697_4f7d7ce270.jpg  1427391496_ea512cbe7f.jpg#2  0.0  0  3
3  1056338697_4f7d7ce270.jpg  2073964624_52da3a0fc4.jpg#2  0.0  0  3
4  1056338697_4f7d7ce270.jpg  2083434441_a93bc6306b.jpg#2  0.0  0  3


Наборы данных успешно загружены. Вот краткий обзор данных:

**Данные экспертных аннотаций**

  - Данные имеют пять столбцов:
         0: Имя файла изображения.
         1: Идентификатор описания.
         2, 3, 4: Рейтинги экспертов, каждый столбец соответствует рейтингу эксперта по шкале от 1 до 4.

**Данные аннотаций краудсорса**

  - Данные также состоят из пяти столбцов:
         0: Имя файла изображения.
         1: Идентификатор описания.
         2: Процент исполнителей, подтвердивших соответствие текста картинке.
         3: Количество исполнителей, подтвердивших соответствие текста картинке.
         4: Количество исполнителей, подтвердивших несоответствие текста картинке.

Следующие шаги будут включать в себя:

  - Агрегирование оценок экспертов в одно значение для каждой пары изображение-текст.
  - Расчет оценки соответствия краудсорсинговых рейтингов.
  - Объединение обоих для создания единой меры вероятности, представляющей вероятность совпадения текста с изображением.

Реализуем эти шаги, чтобы подготовить данные для дальнейшего использования в модели.

In [4]:
# Function to aggregate expert ratings
def aggregate_expert_ratings(ratings):
    mode_rating = ratings.mode()
    if len(mode_rating) == 1:
        # Return the mode if a clear majority exists
        return mode_rating.iloc[0]
    else:
        # Return NaN if no consensus (all different ratings)
        return np.nan

# Aggregating expert ratings
expert_data['aggregated_rating'] = expert_data[[2, 3, 4]].apply(aggregate_expert_ratings, axis=1)

# Calculating consistency score for crowd data
# Using the percentage of performers who confirmed the match as the consistency score
crowd_data['consistency_score'] = crowd_data[2]

# Dropping rows with no consensus in expert ratings
expert_data.dropna(subset=['aggregated_rating'], inplace=True)

# Displaying the first few rows of the transformed datasets
expert_data.head(), crowd_data.head()

(                           0                            1  2  3  4  \
 0  1056338697_4f7d7ce270.jpg  2549968784_39bfbe44f9.jpg#2  1  1  1   
 1  1056338697_4f7d7ce270.jpg  2718495608_d8533e3ac5.jpg#2  1  1  2   
 2  1056338697_4f7d7ce270.jpg  3181701312_70a379ab6e.jpg#2  1  1  2   
 3  1056338697_4f7d7ce270.jpg  3207358897_bfa61fa3c6.jpg#2  1  2  2   
 4  1056338697_4f7d7ce270.jpg  3286822339_5535af6b93.jpg#2  1  1  2   
 
    aggregated_rating  
 0                1.0  
 1                1.0  
 2                1.0  
 3                2.0  
 4                1.0  ,
                            0                            1    2  3  4  \
 0  1056338697_4f7d7ce270.jpg  1056338697_4f7d7ce270.jpg#2  1.0  3  0   
 1  1056338697_4f7d7ce270.jpg   114051287_dd85625a04.jpg#2  0.0  0  3   
 2  1056338697_4f7d7ce270.jpg  1427391496_ea512cbe7f.jpg#2  0.0  0  3   
 3  1056338697_4f7d7ce270.jpg  2073964624_52da3a0fc4.jpg#2  0.0  0  3   
 4  1056338697_4f7d7ce270.jpg  2083434441_a93bc6306b.jpg#2  0.

Экспертные и краудсорсинговые данные аннотаций были успешно преобразованы:

**Преобразование данных экспертных аннотаций**:

  - ***Совокупный рейтинг***: большинство голосов было применено для создания единого совокупного рейтинга для каждой пары изображение-текст. В тех случаях, когда консенсус не был найден (все разные рейтинги), эти записи исключались.

**Преобразование данных краудсорс аннотаций**:

  - ***Оценки соответствия***: процент исполнителей, подтвердивших, что текст соответствует изображению, использовался непосредственно в качестве оценки соответствия.

Благодаря этим преобразованиям оба набора данных теперь предоставляют более упрощенную и полезную меру для понимания согласованности текста и изображения. Следующие шаги будут включать нормализацию этих оценок и их объединение для формирования единого набора данных с целевой вероятностью от 0 до 1 для обучения модели.

In [5]:
# Normalizing Expert Aggregated Rating to a scale of 0-1
expert_data['normalized_rating'] = expert_data['aggregated_rating'] / 4  # as the rating scale is 1 to 4

# Combining the scores
# A simple method to combine the scores is to take a weighted average of the expert and crowd scores.
# As suggested, using 0.6 for expert and 0.4 for crowd scores.
combined_score = (
    0.6 * expert_data['normalized_rating'] + 0.4 * crowd_data['consistency_score']
)

# Creating a combined DataFrame
combined_data = pd.DataFrame({
    'ImageID': expert_data[0],
    'DescriptionID': expert_data[1],
    'ExpertRating': expert_data['normalized_rating'],
    'CrowdConsistency': crowd_data['consistency_score'],
    'CombinedScore': combined_score
})

# Displaying the first few rows of the combined data
combined_data.head()

,ImageID,DescriptionID,ExpertRating,CrowdConsistency,CombinedScore
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,0.25,1.0,0.55
1,1056338697_4f7d7ce270.jpg,2718495608_d8533e3ac5.jpg#2,0.25,0.0,0.15
2,1056338697_4f7d7ce270.jpg,3181701312_70a379ab6e.jpg#2,0.25,0.0,0.15
3,1056338697_4f7d7ce270.jpg,3207358897_bfa61fa3c6.jpg#2,0.50,0.0,0.30
4,1056338697_4f7d7ce270.jpg,3286822339_5535af6b93.jpg#2,0.25,0.0,0.15


Завершены нормализация и объединение экспертных и краудсорсинговых оценок. Вот сводка объединенных данных:

- ***ImageID***: идентификатор изображения.
- ***DescriptionID***: идентификатор конкретного описания, связанного с изображением.
- ***ExpertRating***: нормализованный рейтинг экспертов по шкале от 0 до 1.
- ***CrowdConsistency***: оценка соответствия краудсорсинговых данных.
- ***CombinedScore***: средневзвешенное значение ExpertRating и CrowdConsistency с весами 0,6 и 0,4 соответственно.

Комбинированный балл теперь предоставляет единую меру по шкале от 0 до 1, представляющую вероятность того, что изображение соответствует текстовому описанию. Эту оценку можно использовать в качестве целевой переменной для обучения модели, прогнозирующей согласованность текста и изображения.

# Проверка данных

In [6]:
# Load the dataset
file_path = '/content/drive/My Drive/train_dataset.csv'
train_dataset = pd.read_csv(file_path)

# Display the first few rows of the dataset and its basic info
print(train_dataset.info())
print()
print(train_dataset.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5822 entries, 0 to 5821
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image       5822 non-null   object
 1   query_id    5822 non-null   object
 2   query_text  5822 non-null   object
dtypes: object(3)
memory usage: 136.6+ KB
None

                       image                     query_id  \
0  1056338697_4f7d7ce270.jpg  2549968784_39bfbe44f9.jpg#2   
1  1262583859_653f1469a9.jpg  2549968784_39bfbe44f9.jpg#2   
2  2447284966_d6bbdb4b6e.jpg  2549968784_39bfbe44f9.jpg#2   
3  2549968784_39bfbe44f9.jpg  2549968784_39bfbe44f9.jpg#2   
4  2621415349_ef1a7e73be.jpg  2549968784_39bfbe44f9.jpg#2   

                                          query_text  
0  A young child is wearing blue goggles and sitt...  
1  A young child is wearing blue goggles and sitt...  
2  A young child is wearing blue goggles and sitt...  
3  A young child is wearing blue goggles and sitt...  
4  A

Файл train_dataset.csv содержит 5822 записи и состоит из трех столбцов:

- **image**: этот столбец содержит имена файлов изображений.
- **query_id**: этот столбец имеет уникальный идентификатор для каждого описания в формате <имя файла изображения>#<порядковый номер описания>.
- **query_text**: этот столбец содержит текстовое описание, связанное с каждым изображением.

Из первых нескольких строк очевидно, что несколько строк могут относиться к одному и тому же изображению, но с разными описаниями, или одно и то же описание может повторяться на разных изображениях.

Учитывая поставленную задачу, значительная часть очистки данных будет включать проверку столбца query_text на предмет любых признаков ограниченного контента, особенно описаний с участием детей до 16 лет. Нам нужно будет составить список ключевых слов, которые, вероятно, являются индикаторами такого контента, и отфильтровать все строки, соответствующие этим критериям.

Давайте предположим, что это упрощенный список ключевых слов, которые могут быть связаны с детьми. В реальном сценарии нам нужен более полный и конфиденциальный список, возможно, проверенный командой юристов.

Примеры ключевых слов: ["child", "children", "kid", "boy", "girl", "school", "playground"]

Затем мы отфильтруем описания с помощью ключевых слов. Мы пройдемся по столбцу query_text и удалим все строки, содержащие эти ключевые слова. Этот процесс можно усовершенствовать с помощью регулярного выражения для более детального сопоставления.

In [7]:
# Step 1: Define a simple keyword list for demonstration purposes
keywords = ["child", "children", "kid", "boy", "girl", "school", "playground"]

# Step 2: Filter out descriptions containing these keywords
# Convert query_text to lower case and then check if any of the keywords are in the query_text
filtered_dataset = train_dataset[~train_dataset['query_text'].str.lower().str.contains('|'.join(keywords))]

# Show the effect of filtering
original_count = train_dataset.shape[0]
filtered_count = filtered_dataset.shape[0]

original_count, filtered_count

(5822, 4321)

Процесс фильтрации на основе ключевых слов сократил набор данных с 5822 записей до 4321 записи. Это означает, что 1501 запись была удалена, поскольку они, вероятно, содержали ссылки на детей или связанные контексты.

Это базовый подход и, вероятно, не исчерпывающий. Он служит отправной точкой, и в реальном приложении потребуется более детальный и всеобъемлющий процесс фильтрации, возможно, с использованием более сложных методов обработки естественного языка и более тщательного списка ключевых слов.

# Векторизация изображений

In [8]:
# Step 1: Load the pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)
model = torch.nn.Sequential(*(list(model.children())[:-1]))

# Check if GPU is available and move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.eval()  # Set the model to evaluation mode

# Define the image transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to load an image, transform it, and extract features using GPU
def vectorize_image(image_path, model, transform, device):
    # Load and transform the image
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add a batch dimension

    # Move image to the device (GPU if available)
    image = image.to(device)

    # Extract features
    with torch.no_grad():  # No need to track gradients for feature extraction
        features = model(image)

    # Convert features to a vector
    return features.squeeze().cpu().detach().numpy()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 167MB/s]


In [10]:
def vectorize_images_in_folder(folder_path, model, transform, device):
    import os

    # List all files in the folder
    files = os.listdir(folder_path)
    vectorized_images = []

    for file in files:
        file_path = os.path.join(folder_path, file)
        # Pass the 'device' argument to the vectorize_image function
        vectorized_image = vectorize_image(file_path, model, transform, device)
        vectorized_images.append((file, vectorized_image))

    return vectorized_images

# Make sure to define the device somewhere in your script:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Path to your train_images folder in Google Drive
folder_path = '/content/drive/My Drive/train_images'

# Vectorize all images in the folder, don't forget to pass the 'device'
vectorized_images = vectorize_images_in_folder(folder_path, model, transform, device)

Чтобы векторизовать изображения с помощью сверточной нейронной сети, такой как ResNet-18, были выполнены следующие шаги:

1. ***Загружена предварительно обученная модель***: ResNet-18, предварительно обученную на большом наборе данных ImageNet.
2. ***Удалены полно-связные слои***: Эти слои обычно находятся в конце сети и предназначены для исходной задачи обучения (например, классификации классов ImageNet). Удалив или пропустив эти слои, мы получаем признаки изображения вместо прогноза класса.
3. ***Обработаны изображения***: Изображения стандартизированы до входного размера, ожидаемого моделью, путем изменения их размера и нормализации.
4. ***Извлечены признаки***: изображения были переданы нейронной сети, чтобы получить векторизованное представление.

# Векторизация текстов

In [5]:
# Load the test_queries.csv file
test_queries_path = '/content/drive/My Drive/test_queries.csv'

# Load the dataset with a specified separator and handling bad lines
try:
    test_queries = pd.read_csv(test_queries_path, sep="|", on_bad_lines='skip')
    loaded_successfully = True
except Exception as e:
    loaded_successfully = False
    error_message = str(e)

# Display the outcome
if loaded_successfully:
    print("Data loaded successfully.")
    print(test_queries.head())
else:
    print("Failed to load data:", error_message)

test_queries.drop["Unnamed"]

Data loaded successfully.
   Unnamed: 0                     query_id  \
0           0  1177994172_10d143cb8d.jpg#0   
1           1  1177994172_10d143cb8d.jpg#1   
2           2  1177994172_10d143cb8d.jpg#2   
3           3  1177994172_10d143cb8d.jpg#3   
4           4  1177994172_10d143cb8d.jpg#4   

                                          query_text  \
0  Two blonde boys , one in a camouflage shirt an...   
1  Two boys are squirting water guns at each other .   
2            Two boys spraying each other with water   
3  Two children wearing jeans squirt water at eac...   
4  Two young boys are squirting water at each oth...   

                       image  
0  1177994172_10d143cb8d.jpg  
1  1177994172_10d143cb8d.jpg  
2  1177994172_10d143cb8d.jpg  
3  1177994172_10d143cb8d.jpg  
4  1177994172_10d143cb8d.jpg  


В процессе загрузки данных было обнаружено, что CSV-файл не имеет ожидаемой структуры, в результате чего один столбец со всеми данными и некоторые строки не совпадают с ожидаемыми полями. Первая строка объединила несколько полей в одну строку, разделенную знаком «|».

Чтобы решить эту проблему, я указал разделитель знаком "|" и данные были успешно загружены. DataFrame "test_queries" теперь правильно отображает отдельные столбцы для query_id, query_text и изображения. Также, в датафрейме был еще один безымянный столбец, вероятно, из индекса или дополнительного разделителя в файле, поэтому он был удален.

Теперь, когда данные структурированы правильно, мы можем приступить к векторизации текста с помощью BERT для генерации эмбеддингов.

In [7]:
# Check if a GPU is available and set it as the device, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Initialize the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

# Ensure the model is in evaluation mode
model.eval()

def vectorize_text(texts, tokenizer, model, device):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded_dict = tokenizer.encode_plus(
                            text,
                            add_special_tokens = True,
                            max_length = 64,
                            pad_to_max_length = True,
                            return_attention_mask = True,
                            return_tensors = 'pt',
                       )

        # Move the tensors to the same device as the model
        input_ids.append(encoded_dict['input_ids'].to(device))
        attention_masks.append(encoded_dict['attention_mask'].to(device))

    # Concatenate the list of tensors into a single tensor
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_masks)

        # Retrieve the last hidden states
        last_hidden_states = outputs.last_hidden_state

        # Move the embeddings back to the CPU for further processing or storage
        last_hidden_states = last_hidden_states.cpu()

    return last_hidden_states

# Get the embeddings for the query_text
texts = test_queries['query_text'].tolist()  # Convert the text column to a list
embeddings = vectorize_text(texts, tokenizer, model, device)

Using device: cuda


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Чтобы векторизировать тексты с помощью BERT из Hugging Face Transformers, были выполнены следующие действия:

1. **Настройка среды**: установлена необходимая библиотека из Hugging Face - transformers.
2. **Загрузка предварительно обученной модели BERT и токенизатора**: Использована предварительно обученная модель BERT, подходящая для наших нужд, вместе с соответствующим токенизатором.
3. **Предварительная обработка и токенизация текста**: Подготовлены текстовые данные для модели, включая их токенизацию и правильное форматирование.
4. **Генерация эмбеддингов**: токенизированный текст был передан модели, чтобы получить эмбеддинги - числовые векторы.